In [ ]:
uc_model_name = dbutils.jobs.taskValues.get(taskKey="log-model-to-mlflow", key="uc_model_name")
model_version = dbutils.jobs.taskValues.get(taskKey="log-model-to-mlflow", key="model_version")

In [ ]:
config = {
            "served_entities": [{
                "entity_name": uc_model_name,
                "entity_version": model_version,
                "min_provisioned_throughput": 0,
                "max_provisioned_throughput": 9500,
                "scale_to_zero_enabled": True
            }],
            "traffic_config": {
                "routes": [{
                    "served_model_name": f"{uc_model_name}-{model_version}",
                    "traffic_percentage": 100
                }]
            }
}

In [ ]:
from mlflow.deployments import get_deploy_client
client = get_deploy_client("databricks")

In [ ]:
def endpoint_exists(uc_model_name):
    try:
        client.get_endpoint(uc_model_name)
        return True
    except Exception as e:
        print({str(e)})
        return False

In [ ]:

if endpoint_exists(uc_model_name):
    endpoint = client.get_endpoint(uc_model_name)
    
    assert endpoint.get('state').get('config_update') != 'IN_PROGRESS', "Endpoint is already being updated"

    endpoint = client.update_endpoint(
            endpoint=uc_model_name,
            config=config,
    )
else:
    endpoint = client.create_endpoint(
        name=uc_model_name,
        config=config
    )
    
    